In [ ]:
"""
Don't run this one. For some reason, it is slow..
"""

In [ ]:
import nltk
from nltk import FreqDist
nltk.download('stopwords') # run this one time

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)
import numpy as np
import re
import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('../../data/processed/yp_leilanis-lahaina-2_rws_1.csv')
# df = pd.read_json('../../data/raw/yp_kimos-maui-lahaina_rws.json',orient='records')
df.head()

In [ ]:
reviews = df['reviews']
reviews.head()

In [ ]:
# function to plot most frequent terms
def freq_words(x, terms = 30):
  all_words = ' '.join([text for text in x])
  all_words = all_words.split()

  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})

  # selecting top 20 most frequent words
  d = words_df.nlargest(columns="count", n = terms) 
  plt.figure(figsize=(20,5))
  ax = sns.barplot(data=d, x= "word", y = "count")
  ax.set(ylabel = 'Count')
  plt.show()

In [ ]:
freq_words(reviews, 20)

In [ ]:
!python -m spacy download en 


In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(texts, tags=['NOUN', 'ADJ']): # filter noun and adjective
       output = []
       for sent in texts:
             doc = nlp(" ".join(sent)) 
             output.append([token.lemma_ for token in doc if token.pos_ in tags])
       return output

In [ ]:
tokenized_reviews = pd.Series(reviews).apply(lambda x: x.split())
print(tokenized_reviews[:2])

In [ ]:
reviews_2 = lemmatization(tokenized_reviews)
print(reviews_2[:2]) # print lemmatized review

In [ ]:
from gensim import corpora
from gensim.corpora import Dictionary

dictionary = corpora.Dictionary(reviews_2)

In [ ]:
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(
    corpus=doc_term_matrix, 
    id2word=dictionary, 
    num_topics=4, 
    random_state=100,
    chunksize=1000, 
    passes=50
)

In [ ]:
from gensim.test.utils import datapath
temp_file = datapath("model_final")
lda_model.save(temp_file)

In [ ]:
lda_model

In [ ]:
lda_model.print_topics()

In [ ]:
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)

In [ ]:
lda_display

In [ ]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [ ]:
doc = "start realli good crab cake ok beet salad even though idea chees minut cold entre know is wait long cold like runner took minut break forgot starv eat anyway that 's me"
tokenized = doc.split()
print(tokenized)
corpus = dictionary.doc2bow(tokenized)
doc_lda = lda_model[corpus]
doc_lda